# Extract the Travel Time for the Locations of Different Properties

## Import Packages

In [19]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict

# Database Connection
from sqlalchemy import create_engine

# File and System Operations
import os
import sys

In [20]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

## Other Setup

### Finding the correctd file directory for the credentials json with the api key

In [21]:
current_dir = os.path.dirname(os.path.abspath("NB03_Extract_Travel_Time_Data.ipynb"))
sys.path.insert(0,os.path.join(current_dir, '..'))

credentials_file_path = os.path.join(current_dir, '..', "credentials.json")

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [22]:
pprint(credentials)

{'api_key': '716417c04acf3944937af85d78775ea0', 'app_id': '27beab81'}


In [44]:
# Input the credentials into the header
headers = {
    "Content-Type": "application/json",
    "X-Application-Id": credentials["app_id"],
    "X-Api-Key": credentials["api_key"]
}

# The payload from the curl request
payload = {
    "arrival_searches": {
        "one_to_many": [
            {
                "id": "Example Search",
                "departure_location_id": "Origin",
                "arrival_location_ids": [
                    "Destination 1", "Destination 2", "Destination 3"
                ],
                "transportation": {
                    "type": "public_transport"
                },
                "travel_time": 10800,
                "arrival_time_period": "weekday_morning",
                "properties": ["travel_time", "distance"]
            }
        ]
    },
    "locations": [
        {
            "id": "Origin", # make this the co-ordinates of bank station
            "coords": {
                "lat": 51.513,
                "lng": -0.088
            }
        },
        {
            "id": "Destination 1", # make this oxford circus station
            "coords": {
                "lat": 51.5152,
                "lng": -0.1419
            }
        },
        {
            "id": "Destination 2", # make this some property in walthamstowe
            "coords": {
                "lat": 51.576702,
                "lng": -0.02847
            }
        },
        {
            "id": "Destination 3", # make this a property in nine elms
            "coords": {
                "lat": 51.5163,
                "lng": -0.1300	
            }
        }
    ]
}

# Make the request
response = requests.post(
    "https://api.traveltimeapp.com/v4/time-filter/fast",
    headers=headers,
    data=json.dumps(payload)
)

# Check the response
print(response.status_code)
pprint(response.json())  # or response.text if not JSON

200
{'results': [{'locations': [{'id': 'Destination 1',
                             'properties': {'distance': 0, 'travel_time': 908}},
                            {'id': 'Destination 2',
                             'properties': {'distance': 0,
                                            'travel_time': 2199}},
                            {'id': 'Destination 3',
                             'properties': {'distance': 0,
                                            'travel_time': 857}}],
              'search_id': 'Example Search',
              'unreachable': []}]}


## Augment Property Data with Commute Times

### Load in the Property Data

In [47]:
filtered_df = pd.read_csv("../data/properties.csv").drop(columns = "Unnamed: 0")

### Extract the Commute Times for Each of these Locations

#### Define a Function that Takes the Property IDs, Latitudes and Longitudes, and Creates a Custome Payload for the TravelTime API

In [ ]:
def create_payload(df: pd.DataFrame, search_id: str="1", transportation_type: str = "public_transport") -> dict:
    """
    Creates a payload dictionary for the TravelTime API using property locations from a DataFrame.
    The payload includes an origin (Bank Station) and destination locations (properties), 
    and sets up the search parameters for a one-to-many public transport commute time query.
    """
    # Define origin (Bank Station - a key commuting hub)
    origin = {
        "id": "Origin",
        "coords": {"lat": 51.513, "lng": -0.088}
    }
    # Ensure the 'id' column is of string type for API compatibility
    df["id"] = df["id"].astype(str)

    # Select and rename latitude/longitude columns for API format
    locations = df[["id", "location.latitude", "location.longitude"]].rename(
        columns={"location.latitude": "lat", "location.longitude": "lng"}
    )

    # Convert DataFrame rows to a list of dicts for each destination
    destinations = locations.to_dict(orient="records")
    destination_locations = [
        {
            "id": d["id"],
            "coords": {"lat": d["lat"], "lng": d["lng"]}
        } for d in destinations
    ]

    # Build the final payload structure for the API request
    payload = {
        "arrival_searches": {
            "one_to_many": [
                {
                    "id": search_id,  # Unique search identifier
                    "departure_location_id": "Origin",  # Start from Bank Station
                    "arrival_location_ids": df["id"].tolist(),  # List of property IDs as destinations
                    "transportation": {"type": transportation_type},  # Mode of transport
                    "travel_time": 10800,  # Max travel time in seconds (3 hours)
                    "arrival_time_period": "weekday_morning",  # Commute time window
                    "properties": ["travel_time", "distance"]  # Data to return
                }
            ]
        },
        "locations": [origin] + destination_locations  # All locations (origin + destinations)
    }

    return payload

In [ ]:
payload_1 = create_payload(df=filtered_df)
pprint(payload_1)

{'arrival_searches': {'one_to_many': [{'arrival_location_ids': ['163628153',
                                                                '163931060',
                                                                '163930976',
                                                                '162597587',
                                                                '163930820',
                                                                '163930736',
                                                                '163930730',
                                                                '163930727',
                                                                '163930721',
                                                                '163649780',
                                                                '163930670',
                                                                '163930667',
                                                                '163930664',

In [64]:
# Make the request
response = requests.post(
    "https://api.traveltimeapp.com/v4/time-filter/fast",
    headers=headers,
    data=json.dumps(payload_1)
)

# Check the response
print(response.status_code)
pprint(response.json())  # or response.text if not JSON

200
{'results': [{'locations': [{'id': '163543943',
                             'properties': {'distance': 0,
                                            'travel_time': 1817}},
                            {'id': '163930658',
                             'properties': {'distance': 0,
                                            'travel_time': 1062}},
                            {'id': '163930607',
                             'properties': {'distance': 0, 'travel_time': 325}},
                            {'id': '162597587',
                             'properties': {'distance': 0,
                                            'travel_time': 1279}},
                            {'id': '163930730',
                             'properties': {'distance': 0,
                                            'travel_time': 1598}},
                            {'id': '163628153',
                             'properties': {'distance': 0,
                                            'travel_time': 2006}

### Convert the Response into a New Dataframe with info on Travel Times and Distances for each Property

In [69]:
results = response.json()["results"][0]["locations"]

# Convert to DataFrame
df_results = pd.DataFrame([
    {
        "id": loc["id"],
        "distance": loc["properties"]["distance"],
        "travel_time": loc["properties"]["travel_time"]
    }
    for loc in results
])

df_results

,id,distance,travel_time
0,163543943,0,1817
1,163930658,0,1062
2,163930607,0,325
3,162597587,0,1279
4,163930730,0,1598
5,163628153,0,2006
6,163931060,0,2860
7,163930625,0,402
8,162289487,0,1390
9,163930643,0,482
